# Scraping dei dati sugli episodi di NCIS

Il presente notebook riguarda un progetto di web scraping per raccogliere e pulire i dati relativi ai personaggi e agli episodi della serie TV americana *NCIS*. Per semplificare il più possibile l'attività di scraping, è stata considerata la sola serie madre, in onda dal 2003 e che, al momento della scrittura di questo notebook, conta 22 stagioni.

In questo notebook saranno presentati i due principali approcci che sono stati adottati. Nella prima sezione i dati saranno raccolti e puliti tramite la libreria Pandas, mentre nella seconda sezione le operazioni di raccolta dei dati saranno svolte tramite BeautifulSoup. Nella terza sezione, saranno illustrate le conclusioni tratte dal confronto dei due approcci.

## Scraping con Pandas

In questa prima sezione ci occuperemo dello scraping dei dati tramite la libreria Pandas, che dispone di una funzione apposita per la lettura e l'importazione dei dati dalle pagine HTML.

In particolare, dopo alcune operazioni preliminari e l'importazione dei dati, le tabelle ottenute saranno concatenate in modo da effettuare le prime operazioni di pulizia. I dati ottenuti e trasformati, poi, saranno memorizzati in due file CSV, uno per i dati sul cast e l'altro per i dati sugli episodi.

### Operazioni preliminari

In questa sottosezione ci occuperemo delle operazioni preliminari di supporto all'importazione dei dati.

Per prima cosa è stato creato il dizionario *series*, in cui saranno memorizzate alcune informazioni utili all'importazione dei dati.

In [1]:
series={}

Alla voce "stagioni" è stata memorizzata la lista delle stagioni complete attualmente uscite della serie. Al momento della stesura di questo notebook sono uscite 21 stagioni, mentre la ventiduesima è ancora in corso.

In [2]:
series["stagioni"]=list(range(1,22))

I dati sul cast e parte dei dati sugli episodi saranno raccolti dal sito del fandom di NCIS (consultabile [a questo link](https://ncis.fandom.com/wiki/NCIS_Database)).

Esplorando le pagine relative agli episodi ([qui](https://ncis.fandom.com/wiki/NCIS_Season_1) la pagina della prima stagione per riferimento), osserviamo che i link stessi a questo tipo di pagine presentano un pattern che dipende dal numero di stagione. Per questo motivo, i link così formattati sono stati salvati nella lista "links_stagioni".

In [3]:
series["links_stagioni"]=[f"https://ncis.fandom.com/wiki/NCIS_Season_{stagione}" for stagione in series["stagioni"]]

La seconda parte dei dati sugli episodi è stata invece raccolta da Wikipedia [a questo link](https://en.wikipedia.org/wiki/List_of_NCIS_episodes).

In questa pagina, le prime due tabelle contengono una panoramica generale delle stagioni uscite e una tabella riassuntiva dei due episodi backdoor usciti per la serie *JAG*. Le posizioni delle tabelle che ci interessano (dalla terza in poi) sono state salvate nella lista "table_positions".

In [4]:
series["table_positions"]=[stagione+1 for stagione in series["stagioni"]]

I dati finora ottenuti sono stati convertiti nel DataFrame mostrato dalla cella seguente.

In [5]:
import pandas as pd

pd.DataFrame(series)

,stagioni,links_stagioni,table_positions
0,1,https://ncis.fandom.com/wiki/NCIS_Season_1,2
1,2,https://ncis.fandom.com/wiki/NCIS_Season_2,3
2,3,https://ncis.fandom.com/wiki/NCIS_Season_3,4
3,4,https://ncis.fandom.com/wiki/NCIS_Season_4,5
4,5,https://ncis.fandom.com/wiki/NCIS_Season_5,6
5,6,https://ncis.fandom.com/wiki/NCIS_Season_6,7
6,7,https://ncis.fandom.com/wiki/NCIS_Season_7,8
7,8,https://ncis.fandom.com/wiki/NCIS_Season_8,9
8,9,https://ncis.fandom.com/wiki/NCIS_Season_9,10
9,10,https://ncis.fandom.com/wiki/NCIS_Season_10,11


### Importazione e concatenazione dei dati

In questa sottosezione ci occuperemo di importare i dati relativi al cast e agli episodi tramite la libreria Pandas.

Il primo passo è stato aggiungere le seguenti chiavi al dizionario *series*:
- La chiave "cast" conterrà le tabelle relative al cast regolare delle stagioni;
- La chiave "episodes" conterrà le tabelle su parte dei dati degli episodi;
- La chiave "writers" conterrà le tabelle sui dati relativi agli autori e ai registi degli episodi.

A tutte le chiavi sono state temporaneamente associate delle liste vuote.

In [6]:
series["cast"]=[]
series["episodes"]=[]
series["writers"]=[]

Per ogni stagione, alle liste sopra citate sono state aggiunte le seguenti tabelle:
- La prima tabella della pagina relativa alla stagione è stata aggiunta alla lista "cast";
- La seconda tabella della stessa pagina è stata aggiunta alla lista "episodes";
- La tabella alla posizione indicata in "table_positions" nella pagina degli episodi su Wikipedia è stata aggiunta alla lista "writers".

In [7]:
for i, season in enumerate(series["stagioni"]):
  series["cast"].append(pd.read_html(series["links_stagioni"][i])[0])
  series["episodes"].append(pd.read_html(series["links_stagioni"][i])[1])
  series["writers"].append(pd.read_html("https://en.wikipedia.org/wiki/List_of_NCIS_episodes")[series["table_positions"][i]])

A tutte le tabelle è stata aggiunta la colonna "Season", che indica la stagione a cui la tabella fa riferimento.

In [8]:
for i, season in enumerate(series["stagioni"]):
  series["cast"][i]["Season"]=season
  series["episodes"][i]["Season"]=season
  series["writers"][i]["Season"]=season

Le tre celle successive mostrano le tabelle così ottenute con i dati della prima stagione.

In [9]:
series["cast"][0]

,Name,Portrayed by,Role,Season
0,Leroy Jethro Gibbs,Mark Harmon,NCIS Special Agent in charge of the main NCIS ...,1
1,Caitlin Todd,Sasha Alexander,A Secret Service Agent previously assigned to ...,1
2,Anthony DiNozzo,Michael Weatherly,The Senior Agent of the main NCIS Major Case R...,1
3,Abigail Sciuto,Pauley Perrette,NCIS's Chief Forensic Scientist.,1
4,Vivian Blackadder,Robyn Lively,Former NCIS Special Agent who was DiNozzo's pa...,1
5,Donald Mallard,David McCallum,NCIS's Chief Medical Examiner.,1


In [10]:
series["episodes"][0]

Episode Number                                        Title  \
           Series Episode                                Title   
0           Pilot       -            Ice Queen (episode) (JAG)   
1           Pilot       -             Meltdown (episode) (JAG)   
2           Pilot       -  NCIS: The Beginning (episode) (JAG)   
3               1       1               Yankee White (episode)   
4               2       2            Hung Out to Dry (episode)   
5               3       3                     Seadog (episode)   
6               4       4              The Immortals (episode)   
7               5       5                  The Curse (episode)   
8               6       6                  High Seas (episode)   
9               7       7                   Sub Rosa (episode)   
10              8       8           Minimum Security (episode)   
11              9       9                Marine Down (episode)   
12             10      10              Left for Dead (episode)   
13             11      11                    Eye Spy (episode)   
14             12      12         My Other Left Foot (episode)   
15             13      13         One Shot, One Kill (episode)   
16             14      14         The Good Samaritan (episode)   
17             15      15                     Enigma (episode)   
18             16      16                 Bête Noire (episode)   
19             17      17     The Truth Is Out There (episode)   
20             18      18                   UnSEALeD (episode)   
21             19      19           Dead Man Talking (episode)   
22             20      20                    Missing (episode)   
23             21      21             Split Decision (episode)   
24             22      22                A Weak Link (episode)   
25             23      23                   Reveille (episode)   

                Airdate                                              Notes  \
                Airdate                                              Notes   
0       April 22, 2003.  NCIS investigate the death of a former JAG off...   
1       April 29, 2003.  As the NCIS team investigate Lieutenant Singer...   
2     October 21, 2003.  A modified version of the two pilot episodes, ...   
3   September 23, 2003.  NCIS investigate the death of a Navy Commander...   
4   September 30, 2003.  The NCIS team investigate the death of a Marin...   
5      October 7, 2003.  The bodies of two drug dealers and a Navy Comm...   
6     October 14, 2003.  The NCIS team investigate when the body of a y...   
7     October 28, 2003.  The mummified remains of a Navy Lieutenant are...   
8     November 4, 2003.  A former member of the Major Case Response Tea...   
9    November 18, 2003.  When a badly disfigured body is discovered on ...   
10   November 25, 2003.  A Petty Officer's death and the discovery that...   
11   December 16, 2003.  The team investigate when a supposedly dead Ma...   
12     January 6, 2004.  A woman digs herself out of her grave in Rock ...   
13    January 13, 2004.  NCIS investigate when the body of a Navy Comma...   
14    February 3, 2004.  NCIS investigate when the single leg of a Mari...   
15   February 10, 2004.  NCIS race to stop a sniper from targeting and ...   
16   February 17, 2004.  NCIS investigate the death of a Commander in G...   
17   February 24, 2004.  Gibbs investigates when he discovers that his ...   
18       March 2, 2004.  NCIS face a battle on their hands when an unkn...   
19      March 16, 2004.  NCIS investigate the death of a Petty Officer ...   
20       April 6, 2004.  NCIS investigate when a former Navy SEAL convi...   
21      April 27, 2004.  NCIS investigate when Agent Christopher Pacci ...   
22         May 4, 2004.  NCIS are called to investigate the disappearan...   
23        May 11, 2004.  The NCIS investigate the death of a Marine and...   
24        May 18, 2004.  NCIS investigate the death of a Lieutenant and...   
25         May 2, 2004.  The mysterious terroris

In [11]:
series["writers"][0]

,No. overall,No. in season,Title,Directed by,Written by,Original release date,Prod. code,U.S. viewers (millions),Season
0,1,1,"""Yankee White""",Donald P. Bellisario,Teleplay by : Donald P. Bellisario Story by : ...,"September 23, 2003",101,13.04[6],1
1,2,2,"""Hung Out to Dry""",Alan J. Levi,Don McGill,"September 30, 2003",102,12.08[7],1
2,3,3,"""Seadog""",Bradford May,John C. Kelley & Donald P. Bellisario,"October 7, 2003",103,11.26[8],1
3,4,4,"""The Immortals""",Alan J. Levi,Darcy Meyers,"October 14, 2003",104,11.70[9],1
4,5,5,"""The Curse""",Terrence O'Hara,Teleplay by : Don McGill & Jeff Vlaming & Dona...,"October 28, 2003",105,13.50[10],1
5,6,6,"""High Seas""",Dennis Smith,Teleplay by : Jeff Vlaming & Larry Moskowitz S...,"November 4, 2003",106,11.77[11],1
6,7,7,"""Sub Rosa""",Michael Zinberg,George Schenck & Frank Cardea,"November 18, 2003",107,13.21[12],1
7,8,8,"""Minimum Security""",Ian Toynton,"Philip DeGuere, Jr. & Donald P. Bellisario","November 25, 2003",108,12.71[13],1
8,9,9,"""Marine Down""",Dennis Smith,John C. Kelley,"December 16, 2003",109,12.03[14],1
9,10,10,"""Left for Dead""",James Whitmore Jr.,Teleplay by : Don McGill & Donald P. Bellisari...,"January 6, 2004",110,14.51[15],1


I dati sono stati poi concatenati nei DataFrame *df_cast*, *df_episodes_1* e *df_episodes_2*, di cui si mostrano le prime dieci righe. La pulizia di questi dati sarà affrontata nella prossima sottosezione.

In [12]:
df_cast=pd.concat(series["cast"])
df_cast.head(10)

,Name,Portrayed by,Role,Season,Occupation
0,Leroy Jethro Gibbs,Mark Harmon,NCIS Special Agent in charge of the main NCIS ...,1,NaN
1,Caitlin Todd,Sasha Alexander,A Secret Service Agent previously assigned to ...,1,NaN
2,Anthony DiNozzo,Michael Weatherly,The Senior Agent of the main NCIS Major Case R...,1,NaN
3,Abigail Sciuto,Pauley Perrette,NCIS's Chief Forensic Scientist.,1,NaN
4,Vivian Blackadder,Robyn Lively,Former NCIS Special Agent who was DiNozzo's pa...,1,NaN
5,Donald Mallard,David McCallum,NCIS's Chief Medical Examiner.,1,NaN
0,Leroy Jethro Gibbs,Mark Harmon,NaN,2,NCIS Special Agent in charge of the NCIS Major...
1,Caitlin Todd,Sasha Alexander,NaN,2,NCIS Special Agent.
2,Anthony DiNozzo,Michael Weatherly,NaN,2,Senior Special Agent and second-in-command of ...
3,Abigail Sciuto,Pauley Perrette,NaN,2,NCIS's Chief Forensic Scientist.


In [13]:
df_episodes_1=pd.concat(series["episodes"])
df_episodes_1.head(10)

Episode Number                                        Title  \
          Series Episode                                Title   
0          Pilot       -            Ice Queen (episode) (JAG)   
1          Pilot       -             Meltdown (episode) (JAG)   
2          Pilot       -  NCIS: The Beginning (episode) (JAG)   
3              1       1               Yankee White (episode)   
4              2       2            Hung Out to Dry (episode)   
5              3       3                     Seadog (episode)   
6              4       4              The Immortals (episode)   
7              5       5                  The Curse (episode)   
8              6       6                  High Seas (episode)   
9              7       7                   Sub Rosa (episode)   

               Airdate                                              Notes  \
               Airdate                                              Notes   
0      April 22, 2003.  NCIS investigate the death of a former JAG off...   
1      April 29, 2003.  As the NCIS team investigate Lieutenant Singer...   
2    October 21, 2003.  A modified version of the two pilot episodes, ...   
3  September 23, 2003.  NCIS investigate the death of a Navy Commander...   
4  September 30, 2003.  The NCIS team investigate the death of a Marin...   
5     October 7, 2003.  The bodies of two drug dealers and a Navy Comm...   
6    October 14, 2003.  The NCIS team investigate when the body of a y...   
7    October 28, 2003.  The mummified remains of a Navy Lieutenant are...   
8    November 4, 2003.  A former member of the Major Case Response Tea...   
9   November 18, 2003.  When a badly disfigured body is discovered on ...   

  Season Episode Number Summary Production  
                 Season Summary       Code  
0      1            NaN     NaN        NaN  
1      1            NaN     NaN        NaN  
2      1            NaN     NaN        NaN  
3      1            NaN     NaN        NaN  
4      1            NaN     NaN        NaN  
5      1            NaN     NaN        NaN  
6      1            NaN     NaN        NaN  
7      1            NaN     NaN        NaN  
8      1            NaN     NaN        NaN  
9      1            NaN     NaN        NaN

In [14]:
df_episodes_2=pd.concat(series["writers"])
df_episodes_2.head(10)

,No. overall,No. in season,Title,Directed by,Written by,Original release date,Prod. code,U.S. viewers (millions),Season
0,1,1,"""Yankee White""",Donald P. Bellisario,Teleplay by : Donald P. Bellisario Story by : ...,"September 23, 2003",101,13.04[6],1
1,2,2,"""Hung Out to Dry""",Alan J. Levi,Don McGill,"September 30, 2003",102,12.08[7],1
2,3,3,"""Seadog""",Bradford May,John C. Kelley & Donald P. Bellisario,"October 7, 2003",103,11.26[8],1
3,4,4,"""The Immortals""",Alan J. Levi,Darcy Meyers,"October 14, 2003",104,11.70[9],1
4,5,5,"""The Curse""",Terrence O'Hara,Teleplay by : Don McGill & Jeff Vlaming & Dona...,"October 28, 2003",105,13.50[10],1
5,6,6,"""High Seas""",Dennis Smith,Teleplay by : Jeff Vlaming & Larry Moskowitz S...,"November 4, 2003",106,11.77[11],1
6,7,7,"""Sub Rosa""",Michael Zinberg,George Schenck & Frank Cardea,"November 18, 2003",107,13.21[12],1
7,8,8,"""Minimum Security""",Ian Toynton,"Philip DeGuere, Jr. & Donald P. Bellisario","November 25, 2003",108,12.71[13],1
8,9,9,"""Marine Down""",Dennis Smith,John C. Kelley,"December 16, 2003",109,12.03[14],1
9,10,10,"""Left for Dead""",James Whitmore Jr.,Teleplay by : Don McGill & Donald P. Bellisari...,"January 6, 2004",110,14.51[15],1


### Pulizia dei dati

Questa sottosezione sarà dedicata alla pulizia dei dati ottenuti nella sottosezione precedente. La prima parte è dedicata ai dati sul cast, mentre la seconda parte è dedicata alla pulizia e all'unione dei dati sugli episodi

#### Il dataset *cast_pandas*

Visualizziamo le prime dieci righe dei dati relativi al cast.

In [15]:
df_cast.head(10)

,Name,Portrayed by,Role,Season,Occupation
0,Leroy Jethro Gibbs,Mark Harmon,NCIS Special Agent in charge of the main NCIS ...,1,NaN
1,Caitlin Todd,Sasha Alexander,A Secret Service Agent previously assigned to ...,1,NaN
2,Anthony DiNozzo,Michael Weatherly,The Senior Agent of the main NCIS Major Case R...,1,NaN
3,Abigail Sciuto,Pauley Perrette,NCIS's Chief Forensic Scientist.,1,NaN
4,Vivian Blackadder,Robyn Lively,Former NCIS Special Agent who was DiNozzo's pa...,1,NaN
5,Donald Mallard,David McCallum,NCIS's Chief Medical Examiner.,1,NaN
0,Leroy Jethro Gibbs,Mark Harmon,NaN,2,NCIS Special Agent in charge of the NCIS Major...
1,Caitlin Todd,Sasha Alexander,NaN,2,NCIS Special Agent.
2,Anthony DiNozzo,Michael Weatherly,NaN,2,Senior Special Agent and second-in-command of ...
3,Abigail Sciuto,Pauley Perrette,NaN,2,NCIS's Chief Forensic Scientist.


Possiamo osservare la presenza di valori vuoti nelle colonne "Role" e "Occupation". Contiamo questo tipo di valori su tutte le colonne.

In [16]:
df_cast.isna().sum()

,0
Name,0
Portrayed by,0
Role,41
Season,0
Occupation,125


Dal conteggio dei valori vuoti e da un'ulteriore esplorazione delle tabelle è emerso che le colonne "Role" e "Occupation" indicano lo stesso tipo di dato, ossia il ruolo svolto dal personaggio nella stagione. Dopo aver sostituito i valori mancanti con una stringa vuota, le due colonne sono state unite nella colonna "Role".

In [17]:
df_cast["Role"]=df_cast["Role"].fillna("")+df_cast["Occupation"].fillna("")
df_cast.isna().sum()

,0
Name,0
Portrayed by,0
Role,0
Season,0
Occupation,125


Dalla tabella emergono inoltre delle discrepanze nel nome dei personaggi e degli attori. Per visualizzarle, i dati sono stati raggruppati in base alle colonne "Name" e "Portrayed by".

In [18]:
df_cast.groupby(["Name","Portrayed by"]).count()["Season"]

,,Season
Name,Portrayed by,
Abby Sciuto,Pauley Perrette,1
Abigail Sciuto,Pauley Perrette,14
Alden Parker,Gary Cole,3
Alexandra Quinn,Jennifer Esposito,1
Anthony DiNozzo,Michael Weatherly,4
Anthony DiNozzo Junior,Michael Weatherly,9
Caitlin Todd,Sasha Alexander,2
Clayton Reeves,Duane Henry,2
Donald Mallard,David McCallum,20


Osserviamo che le discrepanze nei dati sono dovute ai nomi dei personaggi riportati in modo diverso (come nel caso di Abby Sciuto) o ad annotazioni aggiuntive (come nel caso di Donald Mallard). Queste annotazioni sono state eliminate sostituendo i valori corrispondenti nelle colonne "Name" e "Portrayed by".

In [19]:
df_cast["Name"]=df_cast["Name"].replace({
    "Abigail Sciuto":"Abby Sciuto",
    "Anthony DiNozzo":"Tony DiNozzo",
    "Anthony DiNozzo Junior":"Tony DiNozzo",
    "Donald Mallard (credit only for episodes 1–2; stand-in: episode 2)":"Donald Mallard"
})

In [20]:
df_cast["Portrayed by"]=df_cast["Portrayed by"].replace({
    "Maria Bello (episode 1–8 only)":"Maria Bello",
    "Sean Murray (actor)":"Sean Murray"
})

Il raggruppamento dei dati con i valori corretti è riportato nella cella sottostante.

In [21]:
df_cast.groupby(["Name","Portrayed by"]).count()["Season"]

,,Season
Name,Portrayed by,
Abby Sciuto,Pauley Perrette,15
Alden Parker,Gary Cole,3
Alexandra Quinn,Jennifer Esposito,1
Caitlin Todd,Sasha Alexander,2
Clayton Reeves,Duane Henry,2
Donald Mallard,David McCallum,21
Eleanor Bishop,Emily Wickersham,8
Jacqueline Sloane,Maria Bello,4
James Palmer,Brian Dietzen,12


Il dataset così ottenuto, ad eccezione della colonna "Occupation" ora superflua, è stato salvato nel file CSV *cast_pandas.csv*.

In [22]:
df_cast[["Name","Portrayed by","Role","Season"]].to_csv("cast_pandas.csv", index=False)

#### Il dataset *episode_pandas*

Visualizziamo le prime dieci righe dei dataframe *df_episodes_1* e *df_episodes_2*.

In [23]:
df_episodes_1.head(10)

Episode Number                                        Title  \
          Series Episode                                Title   
0          Pilot       -            Ice Queen (episode) (JAG)   
1          Pilot       -             Meltdown (episode) (JAG)   
2          Pilot       -  NCIS: The Beginning (episode) (JAG)   
3              1       1               Yankee White (episode)   
4              2       2            Hung Out to Dry (episode)   
5              3       3                     Seadog (episode)   
6              4       4              The Immortals (episode)   
7              5       5                  The Curse (episode)   
8              6       6                  High Seas (episode)   
9              7       7                   Sub Rosa (episode)   

               Airdate                                              Notes  \
               Airdate                                              Notes   
0      April 22, 2003.  NCIS investigate the death of a former JAG off...   
1      April 29, 2003.  As the NCIS team investigate Lieutenant Singer...   
2    October 21, 2003.  A modified version of the two pilot episodes, ...   
3  September 23, 2003.  NCIS investigate the death of a Navy Commander...   
4  September 30, 2003.  The NCIS team investigate the death of a Marin...   
5     October 7, 2003.  The bodies of two drug dealers and a Navy Comm...   
6    October 14, 2003.  The NCIS team investigate when the body of a y...   
7    October 28, 2003.  The mummified remains of a Navy Lieutenant are...   
8    November 4, 2003.  A former member of the Major Case Response Tea...   
9   November 18, 2003.  When a badly disfigured body is discovered on ...   

  Season Episode Number Summary Production  
                 Season Summary       Code  
0      1            NaN     NaN        NaN  
1      1            NaN     NaN        NaN  
2      1            NaN     NaN        NaN  
3      1            NaN     NaN        NaN  
4      1            NaN     NaN        NaN  
5      1            NaN     NaN        NaN  
6      1            NaN     NaN        NaN  
7      1            NaN     NaN        NaN  
8      1            NaN     NaN        NaN  
9      1            NaN     NaN        NaN

In [24]:
df_episodes_2.head(10)

,No. overall,No. in season,Title,Directed by,Written by,Original release date,Prod. code,U.S. viewers (millions),Season
0,1,1,"""Yankee White""",Donald P. Bellisario,Teleplay by : Donald P. Bellisario Story by : ...,"September 23, 2003",101,13.04[6],1
1,2,2,"""Hung Out to Dry""",Alan J. Levi,Don McGill,"September 30, 2003",102,12.08[7],1
2,3,3,"""Seadog""",Bradford May,John C. Kelley & Donald P. Bellisario,"October 7, 2003",103,11.26[8],1
3,4,4,"""The Immortals""",Alan J. Levi,Darcy Meyers,"October 14, 2003",104,11.70[9],1
4,5,5,"""The Curse""",Terrence O'Hara,Teleplay by : Don McGill & Jeff Vlaming & Dona...,"October 28, 2003",105,13.50[10],1
5,6,6,"""High Seas""",Dennis Smith,Teleplay by : Jeff Vlaming & Larry Moskowitz S...,"November 4, 2003",106,11.77[11],1
6,7,7,"""Sub Rosa""",Michael Zinberg,George Schenck & Frank Cardea,"November 18, 2003",107,13.21[12],1
7,8,8,"""Minimum Security""",Ian Toynton,"Philip DeGuere, Jr. & Donald P. Bellisario","November 25, 2003",108,12.71[13],1
8,9,9,"""Marine Down""",Dennis Smith,John C. Kelley,"December 16, 2003",109,12.03[14],1
9,10,10,"""Left for Dead""",James Whitmore Jr.,Teleplay by : Don McGill & Donald P. Bellisari...,"January 6, 2004",110,14.51[15],1


Iniziamo dal dataframe *df_episodes_1*.

Nel corso dell'esplorazione della tabella sono emerse criticità dovute ai nomi delle colonne disposti su doppia riga. Per aggirare il problema, il dataframe è stato salvato in un file CSV temporaneo e poi ricaricato senza i titoli delle colonne.

In [25]:
df_episodes_1.to_csv("tmp.csv", index=False, header=False)
df_episodes_1=pd.read_csv("tmp.csv",header=None)
df_episodes_1

,0,1,2,3,4,5,6,7,8
0,Pilot,-,Ice Queen (episode) (JAG),"April 22, 2003.",NCIS investigate the death of a former JAG off...,1,NaN,NaN,NaN
1,Pilot,-,Meltdown (episode) (JAG),"April 29, 2003.",As the NCIS team investigate Lieutenant Singer...,1,NaN,NaN,NaN
2,Pilot,-,NCIS: The Beginning (episode) (JAG),"October 21, 2003.","A modified version of the two pilot episodes, ...",1,NaN,NaN,NaN
3,1,1,Yankee White (episode),"September 23, 2003.",NCIS investigate the death of a Navy Commander...,1,NaN,NaN,NaN
4,2,2,Hung Out to Dry (episode),"September 30, 2003.",The NCIS team investigate the death of a Marin...,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
466,463,NaN,Strange Invaders (episode),"April 1, 2024",NaN,21,6,After discovering a navy pilot's body riddled ...,2106.0
467,464,NaN,A Thousand Yards (episode),"April 15, 2024",NaN,21,7,NCIS comes under attack by a mysterious enemy ...,2107.0
468,465,NaN,Heartless (episode),"April 22, 2024",NaN,21,8,The NCIS team looks for a motive behind the ki...,2108.0
469,466,NaN,Prime Cut (episode),"April 29, 2024",NaN,21,9,After discovering the remains of a Marine capt...,2109.0


Si osserva che la seconda e la sesta colonna indicano la stessa informazione, ossia il numero di episodio nella stagione, così come la quinta e la settima colonna indicano un riassunto dell'episodio. Queste colonne sono state unite nelle colonne "Episode" e "Synopsis" rispettivamente.

Inoltre, la terza, la quarta e la sesta colonna indicano rispettivamente il titolo, la data della messa in onda e la stagione. Queste colonne sono state rinominate e convertite in stringa.

In [26]:
df_episodes_1["Season"]=df_episodes_1[5].astype(str)
df_episodes_1["Episode"]=df_episodes_1[1].fillna("").astype(str)+df_episodes_1[6].fillna("").astype(str)
df_episodes_1["Title"]=df_episodes_1[2]
df_episodes_1["Airdate"]=df_episodes_1[3]
df_episodes_1["Synopsis"]=df_episodes_1[4].fillna("")+df_episodes_1[7].fillna("")
df_episodes_1=df_episodes_1[["Season","Episode","Title","Airdate","Synopsis"]]
df_episodes_1

,Season,Episode,Title,Airdate,Synopsis
0,1,-,Ice Queen (episode) (JAG),"April 22, 2003.",NCIS investigate the death of a former JAG off...
1,1,-,Meltdown (episode) (JAG),"April 29, 2003.",As the NCIS team investigate Lieutenant Singer...
2,1,-,NCIS: The Beginning (episode) (JAG),"October 21, 2003.","A modified version of the two pilot episodes, ..."
3,1,1,Yankee White (episode),"September 23, 2003.",NCIS investigate the death of a Navy Commander...
4,1,2,Hung Out to Dry (episode),"September 30, 2003.",The NCIS team investigate the death of a Marin...
...,...,...,...,...,...
466,21,6,Strange Invaders (episode),"April 1, 2024",After discovering a navy pilot's body riddled ...
467,21,7,A Thousand Yards (episode),"April 15, 2024",NCIS comes under attack by a mysterious enemy ...
468,21,8,Heartless (episode),"April 22, 2024",The NCIS team looks for a motive behind the ki...
469,21,9,Prime Cut (episode),"April 29, 2024",After discovering the remains of a Marine capt...


Dal dataset sono stati eliminati gli speciali e gli episodi crossover appartenenti ad altre serie, contrassegnati da una stringa nella colonna "Episode".

In [27]:
df_episodes_1['Episode'] = pd.to_numeric(df_episodes_1['Episode'], errors='coerce')
df_episodes_1=df_episodes_1.dropna(subset=['Episode'])
df_episodes_1["Episode"]=df_episodes_1["Episode"].astype(int).astype(str)
df_episodes_1

<ipython-input-27-1dbf67938fbf>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_episodes_1['Episode'] = pd.to_numeric(df_episodes_1['Episode'], errors='coerce')
<ipython-input-27-1dbf67938fbf>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_episodes_1["Episode"]=df_episodes_1["Episode"].astype(int).astype(str)


,Season,Episode,Title,Airdate,Synopsis
3,1,1,Yankee White (episode),"September 23, 2003.",NCIS investigate the death of a Navy Commander...
4,1,2,Hung Out to Dry (episode),"September 30, 2003.",The NCIS team investigate the death of a Marin...
5,1,3,Seadog (episode),"October 7, 2003.",The bodies of two drug dealers and a Navy Comm...
6,1,4,The Immortals (episode),"October 14, 2003.",The NCIS team investigate when the body of a y...
7,1,5,The Curse (episode),"October 28, 2003.",The mummified remains of a Navy Lieutenant are...
...,...,...,...,...,...
466,21,6,Strange Invaders (episode),"April 1, 2024",After discovering a navy pilot's body riddled ...
467,21,7,A Thousand Yards (episode),"April 15, 2024",NCIS comes under attack by a mysterious enemy ...
468,21,8,Heartless (episode),"April 22, 2024",The NCIS team looks for a motive behind the ki...
469,21,9,Prime Cut (episode),"April 29, 2024",After discovering the remains of a Marine capt...


Dalla colonna "Title" sono state eliminate tutte le annotazioni tra parentesi tonde.

In [28]:
import re

df_episodes_1['Title'] = df_episodes_1['Title'].apply(lambda x: re.sub(r'\(.*?\)', '', x).strip())
df_episodes_1

<ipython-input-28-77ec7ee71458>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_episodes_1['Title'] = df_episodes_1['Title'].apply(lambda x: re.sub(r'\(.*?\)', '', x).strip())


,Season,Episode,Title,Airdate,Synopsis
3,1,1,Yankee White,"September 23, 2003.",NCIS investigate the death of a Navy Commander...
4,1,2,Hung Out to Dry,"September 30, 2003.",The NCIS team investigate the death of a Marin...
5,1,3,Seadog,"October 7, 2003.",The bodies of two drug dealers and a Navy Comm...
6,1,4,The Immortals,"October 14, 2003.",The NCIS team investigate when the body of a y...
7,1,5,The Curse,"October 28, 2003.",The mummified remains of a Navy Lieutenant are...
...,...,...,...,...,...
466,21,6,Strange Invaders,"April 1, 2024",After discovering a navy pilot's body riddled ...
467,21,7,A Thousand Yards,"April 15, 2024",NCIS comes under attack by a mysterious enemy ...
468,21,8,Heartless,"April 22, 2024",The NCIS team looks for a motive behind the ki...
469,21,9,Prime Cut,"April 29, 2024",After discovering the remains of a Marine capt...


Infine, al dataset è stata aggiunta la colonna "Episode key", che concatena le colonne "Season" e "Episode" in una stringa.

In [29]:
df_episodes_1["Episode key"] = df_episodes_1["Season"] + "x" + df_episodes_1["Episode"]
df_episodes_1

<ipython-input-29-bbf93e9c45a6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_episodes_1["Episode key"] = df_episodes_1["Season"] + "x" + df_episodes_1["Episode"]


,Season,Episode,Title,Airdate,Synopsis,Episode key
3,1,1,Yankee White,"September 23, 2003.",NCIS investigate the death of a Navy Commander...,1x1
4,1,2,Hung Out to Dry,"September 30, 2003.",The NCIS team investigate the death of a Marin...,1x2
5,1,3,Seadog,"October 7, 2003.",The bodies of two drug dealers and a Navy Comm...,1x3
6,1,4,The Immortals,"October 14, 2003.",The NCIS team investigate when the body of a y...,1x4
7,1,5,The Curse,"October 28, 2003.",The mummified remains of a Navy Lieutenant are...,1x5
...,...,...,...,...,...,...
466,21,6,Strange Invaders,"April 1, 2024",After discovering a navy pilot's body riddled ...,21x6
467,21,7,A Thousand Yards,"April 15, 2024",NCIS comes under attack by a mysterious enemy ...,21x7
468,21,8,Heartless,"April 22, 2024",The NCIS team looks for a motive behind the ki...,21x8
469,21,9,Prime Cut,"April 29, 2024",After discovering the remains of a Marine capt...,21x9


Passiamo ora al dataset *df_episodes_2*.

Di questo dataset sono state conservate le sole colonne relative alla stagione, al numero di episodio, ai registi e agli autori.

In [30]:
df_episodes_2=df_episodes_2[["Season","No. in season","Directed by","Written by"]]
df_episodes_2

,Season,No. in season,Directed by,Written by
0,1,1,Donald P. Bellisario,Teleplay by : Donald P. Bellisario Story by : ...
1,1,2,Alan J. Levi,Don McGill
2,1,3,Bradford May,John C. Kelley & Donald P. Bellisario
3,1,4,Alan J. Levi,Darcy Meyers
4,1,5,Terrence O'Hara,Teleplay by : Don McGill & Jeff Vlaming & Dona...
...,...,...,...,...
5,21,6,Claudia Yarmy,Gina Gold & Aurorae Khoo & Steven D. Binder
6,21,7,Diana Valentine,Christopher J. Waild
7,21,8,Michael Zinberg,Sydney Mitchel & Brendan Fehily
8,21,9,Diana Valentine,Teleplay by : Marco Schnabel Story by : Chad G...


L'unica colonna che necessita di operazioni di pulizia è la colonna "Written by", di cui esploriamo i valori unici.

In [31]:
df_episodes_2["Written by"].unique()

array(['Teleplay by\u200a: Donald P. Bellisario Story by\u200a: Donald P. Bellisario & Don McGill',
       'Don McGill', 'John C. Kelley & Donald P. Bellisario',
       'Darcy Meyers',
       'Teleplay by\u200a: Don McGill & Jeff Vlaming & Donald P. Bellisario Story by\u200a: Donald P. Bellisario',
       'Teleplay by\u200a: Jeff Vlaming & Larry Moskowitz Story by\u200a: Jeff Vlaming',
       'George Schenck & Frank Cardea',
       'Philip DeGuere, Jr. & Donald P. Bellisario', 'John C. Kelley',
       'Teleplay by\u200a: Don McGill & Donald P. Bellisario Story by\u200a: Don McGill',
       'George Schenck & Frank Cardea & Dana Coen', 'Jack Bernstein',
       'Gil Grant', 'Donald P. Bellisario', 'Thomas L. Moran',
       'Bob Gookin', 'Chris Crowe',
       'Teleplay by\u200a: Steven Long Mitchell & Craig W. Van Sickle & Donald P. Bellisario & Gil Grant Story by\u200a: Steven Long Mitchell & Craig W. Van Sickle',
       'Roger Director', 'Jesse Stern & John C. Kelley', 'Frank Military',


Osserviamo l'esistenza di un carattere "\u200a", che rappresenta uno spazio bianco più piccolo del carattere " ". Questo carattere è stato rimosso e sostituito con la stringa vuota.

Da questa colonna emergono poi dei record dove viene fatta una distinzione tra gli autori della sceneggiatura ("Teleplay by") e del soggetto ("Story by"). La prima dicitura è stata sostituita da una stringa vuota, mentre la seconda è stata sostituita dal carattere "&".

In [32]:
df_episodes_2["Written by"]=df_episodes_2["Written by"].str.replace("\u200a","")
df_episodes_2["Written by"]=df_episodes_2["Written by"].str.replace("Teleplay by: ","")
df_episodes_2["Written by"]=df_episodes_2["Written by"].str.replace("Story by:","&")
df_episodes_2["Written by"].unique()

array(['Donald P. Bellisario & Donald P. Bellisario & Don McGill',
       'Don McGill', 'John C. Kelley & Donald P. Bellisario',
       'Darcy Meyers',
       'Don McGill & Jeff Vlaming & Donald P. Bellisario & Donald P. Bellisario',
       'Jeff Vlaming & Larry Moskowitz & Jeff Vlaming',
       'George Schenck & Frank Cardea',
       'Philip DeGuere, Jr. & Donald P. Bellisario', 'John C. Kelley',
       'Don McGill & Donald P. Bellisario & Don McGill',
       'George Schenck & Frank Cardea & Dana Coen', 'Jack Bernstein',
       'Gil Grant', 'Donald P. Bellisario', 'Thomas L. Moran',
       'Bob Gookin', 'Chris Crowe',
       'Steven Long Mitchell & Craig W. Van Sickle & Donald P. Bellisario & Gil Grant & Steven Long Mitchell & Craig W. Van Sickle',
       'Roger Director', 'Jesse Stern & John C. Kelley', 'Frank Military',
       'John C. Kelley & Juan Carlos Coto & Juan Carlos Coto',
       'Chris Crowe & Gil Grant & John C. Kelley', 'Steven Kane',
       'David J. North', 'Christophe

I valori della colonna "Written by" sono stati divisi usando il carattere "&" come delimitatore e le righe sono state "esplose" sulla base delle liste così ottenute.

In [33]:
df_episodes_2["Written by"]=df_episodes_2["Written by"].str.split(" & ")
df_episodes_2=df_episodes_2.explode("Written by")
df_episodes_2

,Season,No. in season,Directed by,Written by
0,1,1,Donald P. Bellisario,Donald P. Bellisario
0,1,1,Donald P. Bellisario,Donald P. Bellisario
0,1,1,Donald P. Bellisario,Don McGill
1,1,2,Alan J. Levi,Don McGill
2,1,3,Bradford May,John C. Kelley
...,...,...,...,...
7,21,8,Michael Zinberg,Brendan Fehily
8,21,9,Diana Valentine,Marco Schnabel
8,21,9,Diana Valentine,Chad Gomez Creasey
8,21,9,Diana Valentine,Marco Schnabel


Dopo aver rimosso i duplicati, i valori di "Written by" sono stati riuniti in una stringa dove i nomi degli autori sono separati dal carattere "&". In questo modo, ci assicuriamo che la colonna "Written by" abbia una lista di autori univoci per ogni episodio.

In [34]:
df_episodes_2=df_episodes_2.drop_duplicates()
df_episodes_2=df_episodes_2.groupby(["Season","No. in season", "Directed by"])["Written by"].apply(" & ".join).reset_index(drop=False)
df_episodes_2

,Season,No. in season,Directed by,Written by
0,1,1,Donald P. Bellisario,Donald P. Bellisario & Don McGill
1,1,2,Alan J. Levi,Don McGill
2,1,3,Bradford May,John C. Kelley & Donald P. Bellisario
3,1,4,Alan J. Levi,Darcy Meyers
4,1,5,Terrence O'Hara,Don McGill & Jeff Vlaming & Donald P. Bellisario
...,...,...,...,...
462,21,6,Claudia Yarmy,Gina Gold & Aurorae Khoo & Steven D. Binder
463,21,7,Diana Valentine,Christopher J. Waild
464,21,8,Michael Zinberg,Sydney Mitchel & Brendan Fehily
465,21,9,Diana Valentine,Marco Schnabel & Chad Gomez Creasey


Anche in questo dataset è stata creata la colonna "Episode key" analoga a *df_episodes_1*. In questo caso, però, le colonne relative alla stagione e all'episodio sono state rimosse.

In [35]:
df_episodes_2["Episode key"] = df_episodes_2["Season"].astype(str) + "x" + df_episodes_2["No. in season"].astype(str)
df_episodes_2=df_episodes_2[["Episode key","Directed by","Written by"]]
df_episodes_2

,Episode key,Directed by,Written by
0,1x1,Donald P. Bellisario,Donald P. Bellisario & Don McGill
1,1x2,Alan J. Levi,Don McGill
2,1x3,Bradford May,John C. Kelley & Donald P. Bellisario
3,1x4,Alan J. Levi,Darcy Meyers
4,1x5,Terrence O'Hara,Don McGill & Jeff Vlaming & Donald P. Bellisario
...,...,...,...
462,21x6,Claudia Yarmy,Gina Gold & Aurorae Khoo & Steven D. Binder
463,21x7,Diana Valentine,Christopher J. Waild
464,21x8,Michael Zinberg,Sydney Mitchel & Brendan Fehily
465,21x9,Diana Valentine,Marco Schnabel & Chad Gomez Creasey


I due dataset sono stati uniti nel dataframe *df_episodes* usando "Episode key" come colonna-chiave per l'unione. La colonna "Episode key" è stata poi rimossa al termine dell'operazione.

In [36]:
df_episodes = df_episodes_1.merge(df_episodes_2, on="Episode key").drop("Episode key",axis=1)
df_episodes

,Season,Episode,Title,Airdate,Synopsis,Directed by,Written by
0,1,1,Yankee White,"September 23, 2003.",NCIS investigate the death of a Navy Commander...,Donald P. Bellisario,Donald P. Bellisario & Don McGill
1,1,2,Hung Out to Dry,"September 30, 2003.",The NCIS team investigate the death of a Marin...,Alan J. Levi,Don McGill
2,1,3,Seadog,"October 7, 2003.",The bodies of two drug dealers and a Navy Comm...,Bradford May,John C. Kelley & Donald P. Bellisario
3,1,4,The Immortals,"October 14, 2003.",The NCIS team investigate when the body of a y...,Alan J. Levi,Darcy Meyers
4,1,5,The Curse,"October 28, 2003.",The mummified remains of a Navy Lieutenant are...,Terrence O'Hara,Don McGill & Jeff Vlaming & Donald P. Bellisario
...,...,...,...,...,...,...,...
462,21,6,Strange Invaders,"April 1, 2024",After discovering a navy pilot's body riddled ...,Claudia Yarmy,Gina Gold & Aurorae Khoo & Steven D. Binder
463,21,7,A Thousand Yards,"April 15, 2024",NCIS comes under attack by a mysterious enemy ...,Diana Valentine,Christopher J. Waild
464,21,8,Heartless,"April 22, 2024",The NCIS team looks for a motive behind the ki...,Michael Zinberg,Sydney Mitchel & Brendan Fehily
465,21,9,Prime Cut,"April 29, 2024",After discovering the remains of a Marine capt...,Diana Valentine,Marco Schnabel & Chad Gomez Creasey


Il dataset ora unito è stato salvato nel file CSV *episodes_pandas.csv*

In [37]:
df_episodes.to_csv("episodes_pandas.csv",index=False)

## Scraping con Beautiful Soup

In questa seconda sezione effettueremo lo scraping degli episodi utilizzando la libreria Beautiful Soup.

Dopo un'analisi delle pagine di interesse allo scopo di definire un piano di azione per la raccolta dei dati, creeremo una classe in grado di memorizzare le informazioni ottenute a partire da funzioni di supporto che si basano su BeautifulSoup per estrarre i dati dalla pagina. Infine, i dati saranno puliti e memorizzati in due file CSV, uno per il cast e l'altro per gli episodi.

### Analisi delle pagine sugli episodi.

Riprendiamo come riferimento il link alla pagina della wiki di NCIS dedicata alla prima stagione ([qui](https://ncis.fandom.com/wiki/NCIS_Season_1) per visitarla).

Nella precedente sezione ci siamo focalizzati sui dati del cast (prima tabella) e degli episodi (seconda tabella). In quest'ultimi, in particolare, notiamo che ogni titolo ha un link che rimanda a una pagina dettagliata riguardo all'episodio.

Visualizziamo la struttura di un campione di queste pagine e verifichiamo se è variata nel tempo. Per questa analisi sono stati scelti i seguenti episodi:
- L'episodio pilota ([*Yankee White*](https://ncis.fandom.com/wiki/Yankee_White_(episode)), Stagione 1, Episodio 1);
- Il centesimo episodio ([*Chimera*](https://ncis.fandom.com/wiki/Chimera_(episode)), Stagione 5, Episodio 6);
- Il duecentesimo episodio ([*Life Before His Eyes*](https://ncis.fandom.com/wiki/Life_Before_His_Eyes_(episode)), Stagione 9, Episodio 14);
- Il trecentesimo episodio ([*Scope*](https://ncis.fandom.com/wiki/Scope_(episode)), Stagione 13, Episodio 18);
- Il quattrocentesimo episodio ([*Everything Starts Somewhere*](https://ncis.fandom.com/wiki/Everything_Starts_Somewhere_(episode)), Stagione 18, Episodio 2);
- L'ultimo episodio andato in onda ([*Humbug*](https://ncis.fandom.com/wiki/Humbug_(NCIS_episode)), Stagione 22, Episodio 9).

Le pagine analizzate, tranne sporadiche eccezioni, presentano tutte la seguente struttura:
- Il titolo dell'episodio in grassetto nel paragrafo introduttivo;
- Un riquadro a destra contenente informazioni su stagione, episodio, autori, registi e data di uscita, oltre che ai titoli dell'episodio precedente e successivo;
- Una sinossi dell'episodio dopo la tabella dei contenuti della pagina;
- Una sezione dedicata al cast dell'episodio, diviso in cast regolare, cast ricorrente e cast ospite.

Questa struttura è utile in quanto non solo ci permette di estrarre le informazioni sugli episodi da una sola fonte, ma dà la possibilità di accedere a più informazioni.

Basandoci su questo, partiremo dal link relativo all'episodio pilota e per ogni episodio andato in onda, si estrarrà dal link alla pagina le informazioni sopra descritte e il link all'episodio successivo.

### L'oggetto Episode

Per raccogliere le informazioni dalla pagina, è stato creata la classe Episode con i seguenti attributi:
- *season*, che memorizza la stagione;
- *episode*, con il numero di episodio della stagione;
- *title* con il titolo dell'episodio;
- *airdate* con la data della messa in onda dell'episodio;
- *director* con il nome del regista;
- *writers* con i nomi degli autori;
- *synopsis* con la sinossi dell'episodio;
- *cast_dict* con un dizionario di informazioni sul cast.

A questi attributi si aggiungono tre metodi:
- Il metodo speciale *_ _ repr _ _* per stampare a video le informazioni sull'episodio;
- Il metodo *episode_table* per scrivere i dati in una riga che poi sarà inserita in un file csv;
- Il metodo *cast_table* per scrivere i dati del cast in un Dataframe Pandas.

In [38]:
class Episode():
  def __init__(self, season, episode, title, airdate, director, writers, synopsis, cast_dict):
    self.season = season
    self.episode = episode
    self.title = title
    self.airdate = airdate
    self.director = director
    self.writers = writers
    self.synopsis = synopsis
    self.cast_dict = cast_dict

  def __repr__(self):
    info = f"Title: {self.title}\n"
    info += f"Season: {self.season}\n"
    info += f"Episode: {self.episode}\n"
    info += f"Airdate: {self.airdate}\n"
    info += f"Director: {self.director}\n"
    info += f"Writers: {self.writers}\n"
    info += f"Synopsis: {self.synopsis}"
    return info

  def episode_table(self):
    return [self.season, self.episode, self.title, self.airdate, self.director, self.writers, self.synopsis]

  def cast_table(self):
    return pd.DataFrame(self.cast_dict)

Le quattro celle sottostanti mostrano un esempio di istanza ottenuta dalla classe Episode da parte dei dati sull'episodio pilota e un test sui suoi metodi.

In [39]:
test_episode = Episode(
    1,
    1,
    "Yankee White",
    "September 23, 2003",
    "Donald Belisario",
    "Donald Belisario & Don McGill",
    "When a Navy Commander collapses and dies after having lunch with the President of the United States on board Air Force, the NCIS team investigate in order to determine if the death was accidental or something far more sinister.",
    {"name":["Leroy Jethro Gibbs", "Caitlin Todd"],
     "portrayed":["Mark Harmon","Sasha Alexander"],
     "role":["NCIS Special Agent","Secret Service Agent"],
     "cast":["regular","guest"]}
)

In [40]:
print(test_episode)

Title: Yankee White
Season: 1
Episode: 1
Airdate: September 23, 2003
Director: Donald Belisario
Writers: Donald Belisario & Don McGill
Synopsis: When a Navy Commander collapses and dies after having lunch with the President of the United States on board Air Force, the NCIS team investigate in order to determine if the death was accidental or something far more sinister.


In [41]:
test_episode.episode_table()

[1,
 1,
 'Yankee White',
 'September 23, 2003',
 'Donald Belisario',
 'Donald Belisario & Don McGill',
 'When a Navy Commander collapses and dies after having lunch with the President of the United States on board Air Force, the NCIS team investigate in order to determine if the death was accidental or something far more sinister.']

In [42]:
test_episode.cast_table()

,name,portrayed,role,cast
0,Leroy Jethro Gibbs,Mark Harmon,NCIS Special Agent,regular
1,Caitlin Todd,Sasha Alexander,Secret Service Agent,guest


### Funzioni di supporto all'estrazione dei dati

Come supporto all'oggetto Episode, definiremo tre funzioni che, a partire dal link dell'episodio, estraggono le informazioni dalla pagina.

Per ottenere questo, è stata installata la libreria Beautiful Soup.

In [43]:
!pip install bs4

La prima funzione definita è *get_title_synopsis* che estrae il titolo dell'episodio (in grassetto nel testo introduttivo) e la sinossi (il primo paragrafo dopo la tabella dei contenuti della pagina e sotto il titolo h2 "Synopsis").

In [44]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

def get_title_synopsis(episode_url):
  episode_bs = BeautifulSoup(urlopen(episode_url),"html.parser")

  if episode_bs.find(id="toc") != None:
    synopsis = episode_bs.find(id="toc").find_next("p").text
  else:
    synopsis = episode_bs.find(id="Synopsis").find_next("p").text

  title = episode_bs.find("b").text
  return title.replace("\n",""), synopsis.replace("\n","")

La seconda funzione è *get_episode_info*, che estrae le informazioni contenute nel riquadro a destra e la lista dei link agli episodi successivi. Qualora questo link non sia disponibile, la funzione restituisce il link alla home del sito.

In [45]:
def get_episode_info(episode_url):
  episode_bs = BeautifulSoup(urlopen(episode_url),"html.parser")

  season = episode_bs.tbody.find(attrs={"data-source":"Season"}).text
  season = ''.join(c for c in season if c.isdigit())

  episode = episode_bs.tbody.find(attrs={"data-source":"Episode"}).text
  episode = int(''.join(c for c in episode if c.isdigit()))

  airdate = episode_bs.find("div",attrs={"data-source":"airdate"}).div.text
  writers = episode_bs.find("div",attrs={"data-source":"writer"}).div.text
  director = episode_bs.find("div",attrs={"data-source":"director"}).div.text

  try:
    next_episode_url = ["https://ncis.fandom.com"+link["href"] for link in episode_bs.find("div", attrs={"data-source": "Next"}).find_all("a")]
  except:
    next_episode_url = "https://ncis.fandom.com"

  return season, episode, airdate, writers, director, next_episode_url

La terza e ultima funzione è *get_episode_cast*, che estrae i dati del cast dell'episodio distinguendoli in regolari, ricorrenti e ospiti. Qualora una di queste categorie non sia disponibile, la funzione passa all'interazione successiva.

In [46]:
def get_episode_cast(episode_url):
  episode_bs = BeautifulSoup(urlopen(episode_url),"html.parser")
  cast_dict = {
      "name":[],
      "actor":[],
      "role":[],
      "cast":[]
  }

  for cast_kind in {"Series_Regulars", "Recurring_Cast", "Guest_Cast"}:
    try:
      table=episode_bs.find(id=cast_kind).find_next("tbody")

      for row in table.find_all("tr"):
        if len(row.find_all("td"))!=0:
          name=row.find_all("td")[0].text
          actor=row.find_all("td")[1].text
          role=row.find_all("td")[2].text

          cast_dict["name"].append(name.replace("\n",""))
          cast_dict["actor"].append(actor.replace("\n",""))
          cast_dict["role"].append(role.replace("\n",""))
          cast_dict["cast"].append(cast_kind)
    except:
      continue

  return cast_dict

Le funzioni appena descritte sono state testate sul link della pagina dedicata all'episodio pilota *Yankee White* e le informazioni ottenute sono state usate per creare un'istanza della classe Episode. Le celle seguenti mostrano il risultato di questo test.

In [47]:
test_episode = "https://ncis.fandom.com/wiki/Yankee_White_(episode)"

title, synopsis = get_title_synopsis(test_episode)
season, episode, airdate, writers, director, next_episode_url = get_episode_info(test_episode)
cast_dict = get_episode_cast(test_episode)
cast_dict["episode_key"] = [f"{season}x{episode}"]*len(cast_dict["name"])

test_episode = Episode(season, episode, title, airdate, director, writers, synopsis, cast_dict)
print(test_episode)

Title: Yankee White
Season: 1
Episode: 1
Airdate: September 23, 2003.
Director: Donald P. Bellisario.
Writers: Donald P. Bellisario- (teleplay).Donald P. Bellisario and Don McGill- (story).
Synopsis: When a Navy Commander collapses and dies after having lunch with the President of the United States on board Air Force, the NCIS team investigate in order to determine if the death was accidental or something far more sinister.


In [48]:
test_episode.episode_table()

['1',
 1,
 'Yankee White',
 'September 23, 2003.',
 'Donald P. Bellisario.',
 'Donald P. Bellisario- (teleplay).Donald P. Bellisario and Don McGill- (story).',
 'When a Navy Commander collapses and dies after having lunch with the President of the United States on board Air Force, the NCIS team investigate in order to determine if the death was accidental or something far more sinister.']

In [49]:
test_episode.cast_table()

,name,actor,role,cast,episode_key
0,Thomas Morrow,Alan Dale,Director of NCIS.,Recurring_Cast,1x1
1,Gerald Jackson,Pancho Demmings,Assistant Medical Examiner for NCIS who helps ...,Guest_Cast,1x1
2,Tobias Fornell,Joe Spano,Agent with the FBI.,Guest_Cast,1x1
3,William Baer,Gerry Becker,United States Secret Service and head of the P...,Guest_Cast,1x1
4,Elmo Poke,Gary Grubbs,"Local Coroner of Wichita, KS and a friend of D...",Guest_Cast,1x1
5,Ray Trapp,Gerald Downey,Commander in the United States Navy and the Fo...,Guest_Cast,1x1
6,Timothy Kerry,Dane Northcutt,Major in the United States Marine Corps.,Guest_Cast,1x1
7,Burger,Michael Adler,President's Physician and also a Captain in th...,Guest_Cast,1x1
8,Leonard Rish,Robert Bagnell,Reporter and also a terrorist who was given th...,Guest_Cast,1x1
9,Carl Pritchard,Dwayne Macopson,Major in the United States Marine Corps and th...,Guest_Cast,1x1


In [50]:
next_episode_url

['https://ncis.fandom.com/wiki/Hung_Out_to_Dry_(episode)']

### Raccolta dei dati

Una volta definiti la classe Episode e le funzioni di supporto, la raccolta dei dati si è svolta secondo i seguenti passi:
1. Nella variabile *current_link* è stato memorizzato il link alla pagina dell'episodio pilota;
2. Per ogni episodio della serie madre, si raccolgono i dati per l'istanza della classe Episode e si seleziona il link dell'episodio successivo;
3. Il passo 2 si ripete finché il prossimo link non corrisponde alla home del sito del fandom.

Per questione di debugging, le informazioni su ogni episodio sono state stampate con il metodo speciale *_ _ repr _ _* definito nella classe Episode.

In [51]:
current_link = "https://ncis.fandom.com/wiki/Yankee_White_(episode)"
next_link = ""

ncis_episodes = []

while next_link != "https://ncis.fandom.com/":
  title, synopsis = get_title_synopsis(current_link)
  season, episode, airdate, writers, director, next_link = get_episode_info(current_link)

  if len(next_link) == 1:
    next_link = next_link[0]
  else:
    season_checks = [(int(get_episode_info(link)[0]), int(get_episode_info(link)[1])) for link in next_link]

    next_link_temp = "https://ncis.fandom.com/"

    for check in season_checks:
      if (check[0] == int(season) and check[1] == int(episode)+1) or (check[0] == int(season)+1 and check[1]==1):
        next_link_temp = next_link[season_checks.index(check)]
        break

    next_link = next_link_temp

  cast_dict = get_episode_cast(current_link)
  cast_dict["episode_key"] = [f"{season}x{episode}"]*len(cast_dict["name"])

  episode = Episode(season, episode, title, airdate, director, writers, synopsis, cast_dict)

  print(episode)
  ncis_episodes.append(episode)

  current_link = next_link

Title: Yankee White
Season: 1
Episode: 1
Airdate: September 23, 2003.
Director: Donald P. Bellisario.
Writers: Donald P. Bellisario- (teleplay).Donald P. Bellisario and Don McGill- (story).
Synopsis: When a Navy Commander collapses and dies after having lunch with the President of the United States on board Air Force, the NCIS team investigate in order to determine if the death was accidental or something far more sinister.
Title: Hung Out to Dry
Season: 1
Episode: 2
Airdate: September 30, 2003.
Director: Alan J. Levi.
Writers: Don McGill.
Synopsis: When a Marine crash-lands through an SUV during a night-training exercise, the initial cause is blamed on a faulty parachute. As the case progresses however, it appears his chute was rigged to fail with the NCIS team suspecting his death could be murder that leads to a drug smuggling ring.
Title: Seadog
Season: 1
Episode: 3
Airdate: October 7, 2003.
Director: Bradford May.
Writers: Donald P. Bellisario and John C. Kelley.
Synopsis: When the

La prossima cella mostra il numero di episodi di cui sono stati raccolti i dati.

In [52]:
len(ncis_episodes)

477

Dopo aver definito le liste *raw_episodes* e *raw_cast* per ogni episodio nella lista sono stati usati i metodi *episode_table* e *cast_table* per estrarre le tabelle di episodi e cast.

In [53]:
raw_episodes = [["Season", "Episode", "Title", "Airdate", "Director", "Writers", "Synopsis"]]
raw_cast = []

for episode in ncis_episodes:
  raw_episodes.append(episode.episode_table())
  raw_cast.append(episode.cast_table())

Tramite la libreria csv, i dati degli episodi sono stati salvati nel file CSV *raw_episodes.csv*

In [54]:
import csv

with open("raw_episodes.csv", "w", newline="") as f:
  writer = csv.writer(f)
  writer.writerows(raw_episodes)

Per quanto riguarda i dati del cast, le tabelle sono state concatenate tramite Pandas prima di essere salvate nel file CSV *raw_cast.csv*.

In [55]:
pd.concat(raw_cast).to_csv("raw_cast.csv", index=False)

### Pulizia dei dati

Ora che sono stati raccolti tutti i dati, possiamo pulirli e salvarli in due file CSV.

Questa sezione sarà divisa in due parti. La prima sottosezione sarà dedicata ai dati sul cast, mentre la seconda sarà dedicata ai dati sugli episodi.

#### Il dataset *cast_bs*

Importiamo i dati grezzi sul cast da *raw_cast.csv* e visualizziamo le prime e le ultime righe della tabella.

In [56]:
df_cast = pd.read_csv("raw_cast.csv")
df_cast

,name,actor,role,cast,episode_key
0,Thomas Morrow,Alan Dale,Director of NCIS.,Recurring_Cast,1x1
1,Gerald Jackson,Pancho Demmings,Assistant Medical Examiner for NCIS who helps ...,Guest_Cast,1x1
2,Tobias Fornell,Joe Spano,Agent with the FBI.,Guest_Cast,1x1
3,William Baer,Gerry Becker,United States Secret Service and head of the P...,Guest_Cast,1x1
4,Elmo Poke,Gary Grubbs,"Local Coroner of Wichita, KS and a friend of D...",Guest_Cast,1x1
...,...,...,...,...,...
8660,James Palmer,Brian Dietzen,Chief Medical Examiner for the NCIS Major Case...,Series_Regulars,22x10
8661,Kasie Hines,Diona Reasonover,NCIS Forensic Specialist.,Series_Regulars,22x10
8662,Jessica Knight,Katrina Law,NCIS Special Agent.,Series_Regulars,22x10
8663,Leon Vance,Rocky Carroll,NCIS Director.,Series_Regulars,22x10


Il primo passo è stato contare i valori mancanti.

In [57]:
df_cast.isna().sum()

,0
name,0
actor,1
role,718
cast,0
episode_key,0


Le due celle seguenti visualizzano nel dettaglio i record dove mancano valori nelle colonne "role" e "actor".

In [58]:
df_cast[df_cast["role"].isna()]

,name,actor,role,cast,episode_key
82,Medical Corpsman,Elizabeth Morehead,NaN,Guest_Cast,1x4
277,Silhouette Man,David Grant Wright,NaN,Guest_Cast,1x15
560,Hotel Clerk,Charles Walker,NaN,Guest_Cast,2x8
700,USN Intelligence Agent,Gorja Max,NaN,Guest_Cast,2x17
1434,Charlie Mills,David Barrera,NaN,Guest_Cast,4x13
...,...,...,...,...,...
8623,Sheila,Alisa Allapach,NaN,Guest_Cast,22x6
8624,Lauren Jacobs,Tabitha Brownstone,NaN,Guest_Cast,22x6
8625,Goon 1,Steven Shelby,NaN,Guest_Cast,22x6
8626,Goon 2,Tim Neff,NaN,Guest_Cast,22x6


In [59]:
df_cast[df_cast["actor"].isna()]

,name,actor,role,cast,episode_key
1196,Abog Galib,NaN,A supposed undercover NCIS Special Agent who i...,Recurring_Cast,3x23


Nel caso della colonna "role", i valori vuoti potrebbero essere dovuti al fatto che gli attori menzionati sono comparse o comunque con un ruolo non fondamentale per l'episodio. Per quanto riguarda "actor", invece, l'assenza di valori potrebbe essere dovuto al fatto che il ruolo non è stato creditato (nel caso dell'episodio in questione, si trattava di una fotografia apparsa per pochi secondi).

In virtù di queste considerazioni, i valori mancanti in "actor" e in "role" sono stati sostituiti con le diciture "Uncredited actor" e "Cameo or Minor Role"

In [60]:
df_cast["actor"]=df_cast["actor"].fillna("Uncredited actor")
df_cast["role"]=df_cast["role"].fillna("Cameo or Minor Role")
df_cast.isna().sum()

,0
name,0
actor,0
role,0
cast,0
episode_key,0


Come avvenuto nello scraping con Pandas, i dati sono stati raggruppati contando gli episodi in base al nome del personaggio e dell'attore che lo interpreta.

In [61]:
df_cast.groupby(["name","actor"])["episode_key"].count().sort_values(ascending=False)

,,episode_key
name,actor,
Donald Mallard,David McCallum,449
Leroy Jethro Gibbs,Mark Harmon,435
James Palmer,Brian Dietzen,362
Leon Vance,Rocky Carroll,359
Abigail Sciuto,Pauley Perrette,353
...,...,...
Greg,Wesley Jonathan,1
Greg Boyen,Tim Ryan,1
Greg Collins,Christopher McDaniel,1


Secondo questo raggruppamento, il personaggio apparso pù spesso è il dottor Donald Mallard, interpretato da David McCallum in 449 episodi. Questa informazione tuttavia non è corretta, in quanto questo record dovrebbe appartenere al personaggio dell'Agente Speciale Timothy McGee interpretato da Sean Murray.

Visualizziamo i raggruppamenti per personaggio e per attore separatamente.

In [62]:
df_cast.groupby("name")["episode_key"].count().sort_values(ascending=False)

,episode_key
name,
Timothy McGee,453
Donald Mallard,449
Leroy Jethro Gibbs,435
James Palmer,362
Leon Vance,359
...,...
Grant Bridges,1
Graves,1
Greg,1


In [63]:
df_cast.groupby("actor")["episode_key"].count().sort_values(ascending=False)

,episode_key
actor,
David McCallum,449
Mark Harmon,436
Brian Dietzen,365
Rocky Carroll,359
Pauley Perrette,353
...,...
George Wyner,1
Georgia Hatzis,1
Georgia Leva,1


Prendendo come riferimento la pulizia dei dati svolta nella sezione dedicata a Pandas, questa discrepanza è stata spiegata con un'annotazione tra parentesi nel nome del personaggio o dell'attore. Date queste considerazioni, quindi, queste annotazioni sono state rimosse.

In [64]:
df_cast['name'] = df_cast['name'].apply(lambda x: re.sub(r'\(.*?\)', '', x).strip())
df_cast['actor'] = df_cast['actor'].apply(lambda x: re.sub(r'\(.*?\)', '', x).strip())
df_cast

,name,actor,role,cast,episode_key
0,Thomas Morrow,Alan Dale,Director of NCIS.,Recurring_Cast,1x1
1,Gerald Jackson,Pancho Demmings,Assistant Medical Examiner for NCIS who helps ...,Guest_Cast,1x1
2,Tobias Fornell,Joe Spano,Agent with the FBI.,Guest_Cast,1x1
3,William Baer,Gerry Becker,United States Secret Service and head of the P...,Guest_Cast,1x1
4,Elmo Poke,Gary Grubbs,"Local Coroner of Wichita, KS and a friend of D...",Guest_Cast,1x1
...,...,...,...,...,...
8660,James Palmer,Brian Dietzen,Chief Medical Examiner for the NCIS Major Case...,Series_Regulars,22x10
8661,Kasie Hines,Diona Reasonover,NCIS Forensic Specialist.,Series_Regulars,22x10
8662,Jessica Knight,Katrina Law,NCIS Special Agent.,Series_Regulars,22x10
8663,Leon Vance,Rocky Carroll,NCIS Director.,Series_Regulars,22x10


Il dataset è stato salvato nel file CSV *cast_bs.csv*.

In [65]:
df_cast.to_csv("cast_bs.csv",index=False)

#### Il dataset *episodes_bs*

In modo analogo, importiamo e visualizziamo i dati contenuti in *raw_episodes.csv*.

In [66]:
df_episodes = pd.read_csv("raw_episodes.csv")
df_episodes

,Season,Episode,Title,Airdate,Director,Writers,Synopsis
0,1,1,Yankee White,"September 23, 2003.",Donald P. Bellisario.,Donald P. Bellisario- (teleplay).Donald P. Bel...,When a Navy Commander collapses and dies after...
1,1,2,Hung Out to Dry,"September 30, 2003.",Alan J. Levi.,Don McGill.,When a Marine crash-lands through an SUV durin...
2,1,3,Seadog,"October 7, 2003.",Bradford May.,Donald P. Bellisario and John C. Kelley.,When the body of a Navy Commander washes up on...
3,1,4,The Immortals,"October 14, 2003.",Alan J. Levi.,Darcy Meyers.,"While on vacation with his friends, a teenager..."
4,1,5,The Curse,"October 28, 2003.",Terrence O'Hara.,"Don McGill, Jeff Vlaming and Donald P. Bellisa...","While in a forest, a hunter discovers a cargo ..."
...,...,...,...,...,...,...,...
472,22,6,Knight & Day,"November 25, 2024.",Rocky Carroll.,Amy Rutberg.,Things become tense when Knight is assigned to...
473,22,7,Hardboiled,"December 2, 2024.",Jose Clemente Hernandez.,Andrew Bartels.,Torres receives intel from a confidential info...
474,22,8,Out of Control,"December 9, 2024.",Diana C. Valentine,Scott Williams & Steven D Binder,NCIS investigates a murder related to a car th...
475,22,9,Humbug,"December 16, 2024.",Lionel Coleman,Christopher J. Waild,When a shocking tell-all threatens to ruin Chr...


Data la struttura delle pagine da cui sono stati raccolti i dati, è certa l'assenza di valori vuoti nel dataset. Notiamo tuttavia che alcuni valori nelle colonne "Director" e "Writers" finiscono con il carattere '.'

In [67]:
df_episodes[df_episodes["Director"].str.endswith(".")].shape[0], df_episodes[df_episodes["Writers"].str.endswith(".")].shape[0]

(470, 469)

Per uniformare questi valori, il punto finale è stato rimosso.

In [68]:
df_episodes['Director'] = df_episodes['Director'].str.rstrip('.')
df_episodes['Writers'] = df_episodes['Writers'].str.rstrip('.')
df_episodes[df_episodes["Director"].str.endswith(".")].shape[0], df_episodes[df_episodes["Writers"].str.endswith(".")].shape[0]

(0, 0)

Per quanto riguarda la colonna "Writers", estraiamo i valori unici.

In [69]:
df_episodes["Writers"].unique()

array(['Donald P. Bellisario- (teleplay).Donald P. Bellisario and Don McGill- (story)',
       'Don McGill', 'Donald P. Bellisario and John C. Kelley',
       'Darcy Meyers',
       'Don McGill, Jeff Vlaming and Donald P. Bellisario- (teleplay).  Donald P. Bellisario- (story)',
       'Jeff Vlaming and Larry Moskowitz- (teleplay).  Jeff Vlaming- (story)',
       'George Schenck and Frank Cardea',
       'Donald P. Bellisario and Philip DeGuere, Jr', 'John C. Kelley',
       'Don McGill and Donald P. Bellisario- (teleplay).  Don McGill (story)',
       'George Schenck, Frank Cardea and Dana Coen', 'Jack Bernstein',
       'Gil Grant', 'Donald P. Bellisario', 'Thomas L. Moran',
       'Bob Gookin', 'Chris Crowe',
       'Steven Long Mitchell, Craig W. Van Sickle, Donald P. Bellisario and Gil Grant- (teleplay).  Steven Long Mitchell and Craig W. Van Sickle- (story)',
       'Roger Director', 'Jesse Stern and John C. Kelley',
       'Frank Military',
       'John C. Kelley and Juan Carlos 

Osserviamo quanto segue:
- Alcuni valori riportano gli autori del soggetto (story), della sceneggiatura (teleplay) o di entrambi (story and teleplay) all'interno delle parentesi;
- Alcuni autori hanno una dicitura "(writer)";
- Il titolo Jr. è separato dal resto del nome con una virgola;
- Laddove la storia è stata scritta da più autori, i nomi sono separati con "and", "&" o con ",".

L'idea è di uniformare questi valori in modo che ogni episodio riporti una lista di autori univoci separati dal carattere "&".

Il primo passo, quindi, è stato rimuovere tutte le diciture e le espressioni regolari superflue.

In [70]:
df_episodes["Writers"]=df_episodes["Writers"].str.replace("story and teleplay","")
df_episodes["Writers"]=df_episodes["Writers"].str.replace("story","")
df_episodes["Writers"]=df_episodes["Writers"].str.replace("teleplay","")
df_episodes["Writers"]=df_episodes["Writers"].str.replace(", Jr"," Jr")

df_episodes["Writers"]=df_episodes["Writers"].str.replace("- ()."," & ")
df_episodes["Writers"]=df_episodes["Writers"].str.replace("- ()","")
df_episodes['Writers'] = df_episodes['Writers'].apply(lambda x: re.sub(r'\(.*?\)', '', x).strip())

df_episodes["Writers"].unique()

array(['Donald P. Bellisario & Donald P. Bellisario and Don McGill',
       'Don McGill', 'Donald P. Bellisario and John C. Kelley',
       'Darcy Meyers',
       'Don McGill, Jeff Vlaming and Donald P. Bellisario &   Donald P. Bellisario',
       'Jeff Vlaming and Larry Moskowitz &   Jeff Vlaming',
       'George Schenck and Frank Cardea',
       'Donald P. Bellisario and Philip DeGuere Jr', 'John C. Kelley',
       'Don McGill and Donald P. Bellisario &   Don McGill',
       'George Schenck, Frank Cardea and Dana Coen', 'Jack Bernstein',
       'Gil Grant', 'Donald P. Bellisario', 'Thomas L. Moran',
       'Bob Gookin', 'Chris Crowe',
       'Steven Long Mitchell, Craig W. Van Sickle, Donald P. Bellisario and Gil Grant &   Steven Long Mitchell and Craig W. Van Sickle',
       'Roger Director', 'Jesse Stern and John C. Kelley',
       'Frank Military',
       'John C. Kelley and Juan Carlos Coto &   Juan Carlos Coto',
       'Chris Crowe, Gil Grant and John C. Kelley', 'Steven Kane',


I separatori "and" e "," sono stati sostituiti dal carattere "&".

In [71]:
df_episodes["Writers"]=df_episodes["Writers"].str.replace(" and "," & ")
df_episodes["Writers"]=df_episodes["Writers"].str.replace(","," & ")
df_episodes["Writers"].unique()

array(['Donald P. Bellisario & Donald P. Bellisario & Don McGill',
       'Don McGill', 'Donald P. Bellisario & John C. Kelley',
       'Darcy Meyers',
       'Don McGill &  Jeff Vlaming & Donald P. Bellisario &   Donald P. Bellisario',
       'Jeff Vlaming & Larry Moskowitz &   Jeff Vlaming',
       'George Schenck & Frank Cardea',
       'Donald P. Bellisario & Philip DeGuere Jr', 'John C. Kelley',
       'Don McGill & Donald P. Bellisario &   Don McGill',
       'George Schenck &  Frank Cardea & Dana Coen', 'Jack Bernstein',
       'Gil Grant', 'Donald P. Bellisario', 'Thomas L. Moran',
       'Bob Gookin', 'Chris Crowe',
       'Steven Long Mitchell &  Craig W. Van Sickle &  Donald P. Bellisario & Gil Grant &   Steven Long Mitchell & Craig W. Van Sickle',
       'Roger Director', 'Jesse Stern & John C. Kelley', 'Frank Military',
       'John C. Kelley & Juan Carlos Coto &   Juan Carlos Coto',
       'Chris Crowe &  Gil Grant & John C. Kelley', 'Steven Kane',
       'David J. North'

Dopo aver diviso i valori in base al carattere "&" e dopo aver esploso le righe del dataset, i valori sono stati puliti dagli spazi bianchi in eccesso.

In [72]:
df_episodes["Writers"]=df_episodes["Writers"].str.split("&")
df_episodes=df_episodes.explode("Writers")
df_episodes["Writers"]=df_episodes["Writers"].str.strip()
df_episodes["Writers"].unique()

array(['Donald P. Bellisario', 'Don McGill', 'John C. Kelley',
       'Darcy Meyers', 'Jeff Vlaming', 'Larry Moskowitz',
       'George Schenck', 'Frank Cardea', 'Philip DeGuere Jr', 'Dana Coen',
       'Jack Bernstein', 'Gil Grant', 'Thomas L. Moran', 'Bob Gookin',
       'Chris Crowe', 'Steven Long Mitchell', 'Craig W. Van Sickle',
       'Roger Director', 'Jesse Stern', 'Frank Military',
       'Juan Carlos Coto', 'Steven Kane', 'David J. North',
       'Christopher Silber', 'Jeffrey Kirkpatrick', 'Joshua Lurie',
       'Lee David Zlotoff', 'Laurence Walsh', 'Steven D. Binder',
       'Richard Arthur', 'Shane Brennan', 'Nell Scovell',
       'Steven Kriozere', 'Robert Palm', 'Christopher Sibler',
       'Alfonso Moreno', 'Dan E. Fesman', 'Reed Steiner', 'Linda Burstyn',
       'Christopher J. Waild', 'Gary Glasberg', 'Nicole Mirante-Matthews',
       'Leon Carroll Jr.', 'Andrew Bartels', 'Scott Williams',
       'Gina Lucita Monreal', 'Allison Abner', 'Bill Nuss',
       'Jennifer C

I duplicati del DataFrame *df_episodes* sono stati rimossi e i valori della colonna "Writers" raggruppati in base all'episodio separando i nomi degli autori con il carattere "&".

In [73]:
df_episodes=df_episodes.drop_duplicates()
df_episodes=df_episodes.groupby(["Season","Episode", "Title","Airdate","Synopsis","Director"])["Writers"].apply(" & ".join).reset_index(drop=False)
df_episodes

,Season,Episode,Title,Airdate,Synopsis,Director,Writers
0,1,1,Yankee White,"September 23, 2003.",When a Navy Commander collapses and dies after...,Donald P. Bellisario,Donald P. Bellisario & Don McGill
1,1,2,Hung Out to Dry,"September 30, 2003.",When a Marine crash-lands through an SUV durin...,Alan J. Levi,Don McGill
2,1,3,Seadog,"October 7, 2003.",When the body of a Navy Commander washes up on...,Bradford May,Donald P. Bellisario & John C. Kelley
3,1,4,The Immortals,"October 14, 2003.","While on vacation with his friends, a teenager...",Alan J. Levi,Darcy Meyers
4,1,5,The Curse,"October 28, 2003.","While in a forest, a hunter discovers a cargo ...",Terrence O'Hara,Don McGill & Jeff Vlaming & Donald P. Bellisario
...,...,...,...,...,...,...,...
472,22,6,Knight & Day,"November 25, 2024.",Things become tense when Knight is assigned to...,Rocky Carroll,Amy Rutberg
473,22,7,Hardboiled,"December 2, 2024.",Torres receives intel from a confidential info...,Jose Clemente Hernandez,Andrew Bartels
474,22,8,Out of Control,"December 9, 2024.",NCIS investigates a murder related to a car th...,Diana C. Valentine,Scott Williams & Steven D Binder
475,22,9,Humbug,"December 16, 2024.",When a shocking tell-all threatens to ruin Chr...,Lionel Coleman,Christopher J. Waild


Il dataset è stato quindi salvato nel file CSV *episodes_bs.csv*.

In [74]:
df_episodes.to_csv("episodes_bs.csv",index=False)

## Un confronto tra i due approcci

Questa sezione conclude il progetto confrontando i due approcci utilizzati per la raccolta dei dati.

Partiamo dai dati sul cast importandoli dai file CSV che abbiamo ottenuto e, più nello specifico, calcoliamo le dimensioni di ogni dataset e l'elenco delle colonne.

In [75]:
df_cast_bs = pd.read_csv("cast_bs.csv")
df_cast_pandas = pd.read_csv("cast_pandas.csv")

In [76]:
df_cast_bs.shape, df_cast_pandas.shape

((8665, 5), (166, 4))

In [77]:
df_cast_bs.columns, df_cast_pandas.columns

(Index(['name', 'actor', 'role', 'cast', 'episode_key'], dtype='object'),
 Index(['Name', 'Portrayed by', 'Role', 'Season'], dtype='object'))

Al di là del diverso nome dato alle colonne, i due dataset presentano le seguenti differenze:
- Il dataset ottenuto con BeautifulSoup è più popolato rispetto allo stesso dataset ottenuto con Pandas, dal momento che nella raccolta dei dati sono stati anche considerati gli attori ricorrenti e gli ospiti;
- Il dataset ottenuto con BeautifulSoup ha due colonne dedicate al riferimento all'attore, una sul tipo di ruolo e l'altra con la stringa identificativa dell'episodio, mentre il dataset ottenuto con Pandas era limitato al cast fisso e pertanto aveva bisogno del numero di stagione come riferimento specifico al personaggio.

In modo simile, otteniamo dimensioni e nomi delle colonne dei dataset relativi agli episodi.

In [78]:
df_episodes_bs = pd.read_csv("episodes_bs.csv")
df_episodes_pandas = pd.read_csv("episodes_pandas.csv")

In [79]:
df_episodes_bs.shape, df_episodes_pandas.shape

((477, 7), (467, 7))

In [80]:
df_episodes_bs.columns, df_episodes_pandas.columns

(Index(['Season', 'Episode', 'Title', 'Airdate', 'Synopsis', 'Director',
        'Writers'],
       dtype='object'),
 Index(['Season', 'Episode', 'Title', 'Airdate', 'Synopsis', 'Directed by',
        'Written by'],
       dtype='object'))

Al di là del nome, le colonne dei due dataset sono le stesse. La differenza più rilevante è dovuta al numero di episodi per cui sono stati ottenuti i dati: nel caso di Pandas ci siamo fermati alle prime 21 stagioni complete, mentre con BeautifulSoup abbiamo ottenuto i dati anche sulla ventiduesima stagione ancora in corso.

Questa considerazione mette in risalto il limite principale di Pandas, che opera solo sulle tabelle in una pagina HTML. Di conseguenza, i dati che sono stati potuti raccogliere sono stati più limitati, al punto che per le informazioni su registi e autori ci si è appoggiati a una seconda fonte. Dall'altra parte, con BeautifulSoup abbiamo potuto estrarre le stesse informazioni da un'unica pagina (quella dell'episodio) e abbiamo ottenuto più dati sul cast, anche se il principale limite è stato in termini di pulizia dei dati e di connessione a Internet.

## Conclusione

In questo progetto sono stati raccolti i dati relativi agli episodi della serie televisiva americana *NCIS*, in onda dal 2003 e che conta 22 stagioni.

Nella prima sezione, è stato utilizzato Pandas per importare e pulire i dati sul cast regolare e sugli episodi delle prime ventuno stagioni. Mentre i dati del cast sono stati ottenuti da un'unica fonte, i dati sugli episodi sono stati ottenuti da due fonti diverse e poi uniti nello stesso dataset.

Nella seconda sezione, utilizzando BeautifulSoup, sono stati importati per ogni episodio i dati sulla puntata e sul cast. I dati sono stati ottenuti definendo l'oggetto Episode e tre funzioni di supporto per la raccolta delle informazioni e hanno riguardati tutti gli episodi attualmente usciti.

La terza sezione è stata dedicata al confronto tra i due approcci utilizzati. Mentre Pandas presenta dei limiti in termini di quali informazioni possono essere importate, BeautifulSoup ha dei limiti solo in termini di pulizia e di connessione a Internet.